In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 221 kB 5.2 MB/s 
     |████████████████████████████████| 2.8 MB 37.5 MB/s 
     |████████████████████████████████| 264 kB 46.0 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 3.3 MB 34.3 MB/s 
     |████████████████████████████████| 1.2 MB 52.6 MB/s 
     |████████████████████████████████| 1.7 MB 36.4 MB/s 
     |████████████████████████████████| 8.0 MB 35.6 MB/s 
     |████████████████████████████████| 124 kB 49.2 MB/s 
     |████████████████████████████████| 895 kB 39.1 MB/s 
     |████████████████████████████████| 50 kB 6.9 MB/s 
     |████████████████████████████████| 636 kB 38.4 MB/s 
     |████████████████████████████████| 133 kB 43.1 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 170 kB 48.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 243 kB 37.5 MB/s 
     |█████████████████

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/news_summary.csv", encoding='latin-1')
df = df[['text','ctext']]
df = df[['ctext','text']].head(1000)
df = df.rename(columns = {'ctext':'input_text' , 'text' : 'target_text'})
df
# df = df[['Plot', 'PlotSummary']].head(500)
# df = df.rename(columns={'Plot' : 'input_text', 'PlotSummary':'target_text'})
# df = df.dropna()
# print(df)

,input_text,target_text
0,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...
1,"From her special numbers to TV?appearances, Bo...",Malaika Arora slammed an Instagram user who tr...
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...
...,...,...
995,"New Delhi, Jul 18 (PTI) The government has scr...",NITI Aayog CEO Amitabh Kant has said the gover...
996,"From a distance, this government-run Sanskrit ...",Government Upper Primary Sanskrit School in Ra...
997,?An 18-year-old son of MHADA vice-president Mi...,An 18-year-old son of Maharashtra Housing and ...
998,"Indian all-rounder Irfan Pathan, on Tuesday, w...",Indian all-rounder Irfan Pathan was slammed on...


In [ ]:
for i in range(1000):
  df.input_text[i] = str(df.input_text[i])
  df.input_text[i] = df.input_text[i].replace("?","") 
  df.input_text[i] = df.input_text[i].replace("/","") 
  df.input_text[i] = df.input_text[i].replace(")","")
  df.input_text[i] = df.input_text[i].replace("(","")
  df.input_text[i] = df.input_text[i].replace("|","")
  df.input_text[i] = df.input_text[i].replace(",","")
  df.input_text[i] = df.input_text[i].replace("'","")
  df.input_text[i] = df.input_text[i].replace("\t","")
  df.input_text[i] = df.input_text[i].replace("\r","") 
  df.input_text[i] = df.input_text[i].replace("\n","")
  df.input_text[i] = df.input_text[i].replace('"',"")
  df.input_text[i] = df.input_text[i].replace('@',"")
  df.input_text[i] = df.input_text[i].replace('#',"")
  df.input_text[i] = df.input_text[i].replace('&',"")
  df.input_text[i] = df.input_text[i].replace('*',"")
  df.input_text[i] = df.input_text[i].replace('!',"")
  df.input_text[i] = df.input_text[i].replace('-',"")
  df.input_text[i] = df.input_text[i].lower()

In [ ]:
for i in range(1000):
  df.target_text[i] = df.target_text[i].replace("?","") 
  df.target_text[i] = df.target_text[i].replace("/","") 
  df.target_text[i] = df.target_text[i].replace(")","")
  df.target_text[i] = df.target_text[i].replace("(","")
  df.target_text[i] = df.target_text[i].replace("|","")
  df.target_text[i] = df.target_text[i].replace(",","")
  df.target_text[i] = df.target_text[i].replace("'","")
  df.target_text[i] = df.target_text[i].replace("\t","")
  df.target_text[i] = df.target_text[i].replace("\r","") 
  df.target_text[i] = df.target_text[i].replace("\n","")
  df.target_text[i] = df.target_text[i].replace('"',"")
  df.target_text[i] = df.target_text[i].replace('@',"")
  df.target_text[i] = df.target_text[i].replace('#',"")
  df.target_text[i] = df.target_text[i].replace('&',"")
  df.target_text[i] = df.target_text[i].replace('*',"")
  df.target_text[i] = df.target_text[i].replace('!',"")
  df.target_text[i] = df.target_text[i].replace('-',"")
  df.target_text[i] = df.target_text[i].lower()

In [ ]:
df['prefix'] = 'summarization' 
df = df[['prefix' , 'input_text' , 'target_text']]
df

,prefix,input_text,target_text
0,summarization,the daman and diu administration on wednesday ...,the administration of union territory daman an...
1,summarization,from her special numbers to tvappearances boll...,malaika arora slammed an instagram user who tr...
2,summarization,the indira gandhi institute of medical science...,the indira gandhi institute of medical science...
3,summarization,lashkaretaibas kashmir commander abu dujana wa...,lashkaretaibas kashmir commander abu dujana wh...
4,summarization,hotels in mumbai and other indian cities are t...,hotels in maharashtra will train their staff t...
...,...,...,...
995,summarization,new delhi jul 18 pti the government has scrapp...,niti aayog ceo amitabh kant has said the gover...
996,summarization,from a distance this governmentrun sanskrit sc...,government upper primary sanskrit school in ra...
997,summarization,an 18yearold son of mhada vicepresident milind...,an 18yearold son of maharashtra housing and ar...
998,summarization,indian allrounder irfan pathan on tuesday was ...,indian allrounder irfan pathan was slammed on ...


In [ ]:
df = df.dropna()
df
from sklearn.model_selection import train_test_split
train , test = train_test_split(df , test_size = 0.2)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [ ]:
train.shape , test.shape

((800, 3), (200, 3))

In [ ]:
import logging
from simpletransformers.t5 import T5Model, T5Args
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Configure the model
model_args = T5Args()
model_args.num_train_epochs = 50
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True
model_args.train_batch_size=12
model_args.eval_batch_size = 8
model_args.max_seq_length = 350
model_args.learning_rate = 2e-5
model_args.preprocess_inputs = True
model_args.early_stopping = True
model_args.max_length = 350
model_args.repetition_penalty = 2.5

model = T5Model(
    "t5",
    "t5-small",
    args=model_args,
    use_cuda = True
)

# Train the model
model.train_model(train, eval_data=test, output_dir = "/content/drive/MyDrive")


Decoding outputs:   0%|          | 0/200 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 1.92546733379364}


Running Epoch 49 of 50:   0%|          | 0/67 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/200 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-small_cached_350800
INFO:simpletransformers.t5.t5_model: Training started


Using Adafactor for T5


Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Running Epoch 0 of 50:   0%|          | 0/67 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
result = model.eval_model(test)

INFO:simpletransformers.t5.t5_utils: Creating features from dataset file at cache_dir/


  0%|          | 0/200 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
INFO:simpletransformers.t5.t5_utils: Saving features into cached file cache_dir/t5-small_cached_350200


Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/25 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/200 [00:00<?, ?it/s]

INFO:simpletransformers.t5.t5_model:{'eval_loss': 1.925534930229187}


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Mon Sep  6 08:58:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    63W / 149W |   7817MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model_args = T5Args()
model_args.num_train_epochs = 10
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir = True
model_args.train_batch_size=12
model_args.eval_batch_size =12
model_args.max_length = 512

model_reloaded = T5Model(
    "t5",
    "outputs/best_model",
    args=model_args
)

In [ ]:
print(
    model_reloaded.predict(
        [
          "We humans always need an escape from the reality once in a while. No better therapy than Maldives is truly a paradise on earth! And if you’re wishing to travel somewhere from india during this COVID situations and restrictions then this is the best option in present. It is open for all the Indians and all the safety measures are well taken care of. There are some points to be noted before plan a trip for Maldives. it’s always better to choose the all inclusive package. Choose a resort which is giving you good inclusions and facilities such as food, water activities, spa, privacy for a honeymoon couple, complementary RTPCR test.You need to have a RTPCR test done before 72 hours of your flight date and get a mandatory negative report! Have a travel insurance and make sure you’ve filled all the documents and have a xerox copies of it too. Reach the airport prior 2 hours of your flight timings for the safer side because check in and immigration might take some time. make sure you choose the earliest flight available of the day as your first day won’t be wasted because after arriving in Maldives airport you’ll have to wait for your boat or sea plane for your resort and almost one hour goes here. Sea planes are available only till 4 pm so if you have to catch a sea plane you need to land on Maldives airport  by 2 pm max! "
        ]
                )
)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3365: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

['ands which is the best option in present. and. Choose a resort which is giving you good inclusions and facilities such as food, water activities and water activities. choose a resort which is giving you good inclusions and facilities such as food and water activities.']


In [ ]:
import pandas as pd
df1 = pd.read_csv("/content/drive/MyDrive/news_summary.csv", encoding='latin-1')
df1 = df1[['text','ctext']]
df1 = df1[['ctext','text']].head(1500)
df1 = df1.rename(columns = {'ctext':'input_text' , 'text' : 'target_text'})
df1

,input_text,target_text
0,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...
1,"From her special numbers to TV?appearances, Bo...",Malaika Arora slammed an Instagram user who tr...
2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...
3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...
4,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...
...,...,...
1495,Protests over the ban on the bull-taming conti...,Amid the ongoing protests over the ban on the ...
1496,The National Investigating Agency (NIA) today ...,The National Investigating Agency (NIA) on Thu...
1497,The sports ministry has refused the Archery As...,The Indian Sports Ministry has decided to post...
1498,NaN,The Akhil Bharat Hindu Mahasabha has decided t...


In [ ]:
df1.input_text[1102]

'former india captain sourav ganguly has already claimed that india can whitewash australia 40 in the upcoming test series but he has also cautioned virat kohli saying the visitors will come hard at him in the first test in pune.click to watch full videokohli had to face a barrage of verbal volleys from the aussies during the 201415 tour down under but that only made him more hungry to score runs and he did so in style slamming four centuries in as many matches.while ganguly feels the sledging wont affect kohli too much but he is certain that steve smith and his team will try and make life as difficult for the indian captain as possible.im not sure its going to make any difference to virat kohli. they targeted him in australia and he came back and played superbly. for me it was the life changing series for kohli and since then hes been a terrific player. they might come back hard at him they will do that in the first test match but theyll have to keep winning and competing as well agai

In [ ]:
df1.input_text[1105]

'in an exclusive interview with india today samajwadi party patriarch mulayam singh yadav today said that it is the congress that stands to benefit from the alliance for uttar pradesh assembly election not his party.congress had no footing in uttar pradesh. this alliance has revived their chances to win some seats. the contest was otherwise between the samajwadi party and the bharatiya janata party bjp he said.talking about the 2012 assembly elections mulayam reassuringly said that he did not regret making akhilesh the chief minister. but added i made akhilesh the chief minister because i knew if i did not he would never become the cm.refuting rumours of an ongoing turmoil in the family and the party he said that those who did not get the ticket were still angry but there was no rift in the party as such.also read: uttar pradesh: polarisation could scupper rahulakhilesh alliance in assembly elections also read: uttar pradesh polls: akhilesh yadav promises 1 lakh jobs in police force wi

In [ ]:
df1.input_text[1111]

'the mumbai customs on friday arrested one air passenger with indian passport with smuggled gold worth rs 91 lakh.\xa0 he was carrying the gold concealed in a steel roller.'

In [ ]:
df1.input_text[1450]

'based on profiling and intelligence air intelligence unit aiu officers intercepted two cases of illegal smuggling of gold and tobacco products valued at around rs 42.5 lakh at mumbai airport.tobacco casetwo passengers vijay kumar natvarlal rupapara and manish kumar mansukhlal khambani holding indian passports on their departure to lubumbashi via addis ababa by ethiopian air flight no. et 641 were intercepted.the said passengers were attempting to export 5869 pouches and 900 tins of tobacco products 185.5 kgs of cut betelnuts 11 pieces of mobile phones and 2 pieces of tablets totally valued at rs 512812 out of india by concealing the same in their 17 checkedinbaggage and 4 hand baggage weighing 597 kgs.the passengers in their statements recorded under sec 108 of the customs act 1962 has admitted knowledge possession carriage concealment and recovery of said goods which were being attempted to smuggle out of india.also read  smugglers use india to ferry iphones to nepal and bangladesha 

In [ ]:
df1.input_text[1234]

'jolly llb 2 starring akshay kumar and huma qureshi in the lead roles is the big bollywood release this week. will the sequel to jolly llb match up to the first film? here is our jolly llb 2 movie review.jolly llb 2 cast: akshay kumar huma qureshi annu kapoor saurabh shuklajolly llb 2 director: subhash kapoorjolly llb 2 rating:\xa0 3.55jolly llb was a jolly good movie. a little film by subhash kapoor jolly llb was shouldered by actors like arshad warsi and saurabh shukla. actors not stars. so when a second film in the franchise is announced and an akshay kumar is attached along with it it sort of brings about mixed feelings in people. on the one hand while a star comes with expectations along theres somewhere a shred of regret at seeing yet another tiny film being hijacked by a star. but kumar doesnt let his stardom overshadow his performance.in the courtroom drama called jolly llb 2 jagdishwar mishra aka jolly akshay kumar is the lead actor. jolly llb 2 is generously peppered with one

In [ ]:
for i in range(len(df1)):
  df1.input_text[i] = str(df1.input_text[i])
  df1.input_text[i] = df1.input_text[i].replace("/","") 
  df1.input_text[i] = df1.input_text[i].replace(")","")
  df1.input_text[i] = df1.input_text[i].replace("(","")
  df1.input_text[i] = df1.input_text[i].replace("|","")
  df1.input_text[i] = df1.input_text[i].replace(",","")
  df1.input_text[i] = df1.input_text[i].replace("'","")
  df1.input_text[i] = df1.input_text[i].replace("\t","")
  df1.input_text[i] = df1.input_text[i].replace("\r","") 
  df1.input_text[i] = df1.input_text[i].replace("\n","")
  df1.input_text[i] = df1.input_text[i].replace('"',"")
  df1.input_text[i] = df1.input_text[i].replace('@',"")
  df1.input_text[i] = df1.input_text[i].replace('#',"")
  df1.input_text[i] = df1.input_text[i].replace('&',"")
  df1.input_text[i] = df1.input_text[i].replace('*',"")
  df1.input_text[i] = df1.input_text[i].replace('!',"")
  df1.input_text[i] = df1.input_text[i].replace('-',"")
  df1.input_text[i] = df1.input_text[i].lower()

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
import torch 
import torch.nn.functional as F

model_name = T5ForConditionalGeneration.from_pretrained("outputs/best_model")
tokenizer = AutoTokenizer.from_pretrained("outputs/best_model")

generator = pipeline('summarization', model=model_name, tokenizer= tokenizer)
print("generator done")
text = "We humans always need an escape from the reality once in a while. No better therapy than Maldives is truly a paradise on earth! And if you’re wishing to travel somewhere from india during this COVID situations and restrictions then this is the best option in present. It is open for all the Indians and all the safety measures are well taken care of. There are some points to be noted before plan a trip for Maldives. it’s always better to choose the all inclusive package. Choose a resort which is giving you good inclusions and facilities such as food, water activities, spa, privacy for a honeymoon couple, complementary RTPCR test.You need to have a RTPCR test done before 72 hours of your flight date and get a mandatory negative report! Have a travel insurance and make sure you’ve filled all the documents and have a xerox copies of it too. Reach the airport prior 2 hours of your flight timings for the safer side because check in and immigration might take some time. make sure you choose the earliest flight available of the day as your first day won’t be wasted because after arriving in Maldives airport you’ll have to wait for your boat or sea plane for your resort and almost one hour goes here. Sea planes are available only till 4 pm so if you have to catch a sea plane you need to land on Maldives airport  by 2 pm max! "
results = generator(text, max_length =300)
result = results[0]['summary_text']
print(result)

Your max_length is set to 1000, but you input_length is only 310. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


generator done
if you’re wishing to travel somewhere from india during this COVID situations and restrictions then this is the best option in present. choose a resort which is giving you good inclusions and facilities such as food, water activities, spa, privacy for a honeymoon couple and complementary RTPCR test. make sure you’ve filled all the documents and have a mandatory negative report.


In [ ]:
!pip install transformers

In [ ]:
!pip install flask-ngrok

In [ ]:
import torch
import flask
import time
from flask import Flask, render_template, request
from flask import request
from transformers import pipeline, AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
import torch.nn as nn
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app) 

@app.route('/', methods=['GET'])
def index():
    return render_template('index.html')

@app.route('/prediction', methods=['POST','GET'])
def prediction() : 
    model_name = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/checkpoint-2747-epoch-41")
    tokenizer = AutoTokenizer.from_pretrained("t5-small")
    generator = pipeline("summarization" ,model=model_name, tokenizer=tokenizer)
    results = generator(str(request.form['text'])) 
    result = results[0]['summary_text']
    return render_template('index.html', result=result)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6645-34-76-41-48.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Sep/2021 16:35:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2021 16:35:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Sep/2021 16:35:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2021 16:35:41] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2021 16:36:41] "POST /prediction HTTP/1.1" 200 -
Your max_length is set to 200, but you input_length is only 47. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
127.0.0.1 - - [06/Sep/2021 16:37:28] "POST /prediction HTTP/1.1" 200 -
127.0.0.1 - - [06/Sep/2021 16:37:48] "POST /prediction HTTP/1.1" 200 -
Token indices sequence length is longer than the specified maximum sequence length for this model (1196 > 512). Running this sequence through the model will result in indexing errors
127.0.0.1 - - [06/Sep/2021 16:39:29] "POST /prediction HTTP/1.1" 200 -
